In [ ]:
ㅎ!pip install huggingface_hub datasets trl peft

In [ ]:
# 사전 설치 : pip install datasets trl  (코랩에서 설치 시 느낌표 추가 : !pip install datsets trl)
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from huggingface_hub import login
from datasets import Dataset
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from trl import SFTTrainer

In [ ]:
# 1. Hugging Face 토큰 설정 및 로그인
hf_token = "inset your key"  # 본인의 Fine-grained 토큰 입력
login(hf_token)

In [ ]:
# 2. 모델과 토크나이저 로드
model_id = "google/gemma-2-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)

# 모델에 필요한 특수 토큰 설정
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token=hf_token,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation="eager"
)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
# 3. 데이터셋 준비 및 토큰화
# CSV 파일 로드
df = pd.read_csv("dataset/indata_kor.csv", encoding="cp949")

In [ ]:
# 결측값 제거 (필요 시)
df = df.dropna()

In [ ]:
# 데이터프레임을 질문-답변 쌍으로 변환 (프롬프트 형식 추가)
qa_pairs = [{"input": f"### 질문: {row['inputs']}", "output": f"### 답변: {row['response']}"} for _, row in df.iterrows()]

print(f"데이터셋 크기: {len(qa_pairs)}개의 질문-답변 쌍")

데이터셋 크기: 32개의 질문-답변 쌍


In [ ]:
# 데이터셋 형식으로 변환
dataset = Dataset.from_list(qa_pairs)

In [ ]:
# 토큰화 함수 정의
def tokenize_function(examples):
    # 질문과 답변을 공백으로 연결 (샘플 데이터에 맞게)
    full_texts = [f"{inp} {out}" for inp, out in zip(examples["input"], examples["output"])]
    tokenized = tokenizer(full_texts, truncation=True, max_length=512, padding="max_length")
    return tokenized

In [ ]:
# 데이터셋 토큰화
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["input", "output"])

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [ ]:
# 4. LoRA 설정
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

In [ ]:
# 모델에 LoRA 적용
model = get_peft_model(model, lora_config)

In [ ]:
# 5. 학습 설정
training_args = TrainingArguments(
    output_dir="./gemma-2-2b-it-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    num_train_epochs=10,
    logging_steps=5,
    save_steps=50,
    optim="adamw_torch",
    fp16=True,
    report_to="none"
)

In [ ]:
# 6. SFTTrainer 설정 및 학습
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    peft_config=lora_config
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Truncating train dataset:   0%|          | 0/32 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# 학습 시작
trainer.train()

Step,Training Loss
5,1.729300
10,0.484200
15,0.442400
20,0.379200
25,0.300300
30,0.287000
35,0.279900
40,0.248200
45,0.220600
50,0.204200


TrainOutput(global_step=80, training_loss=0.346078397333622, metrics={'train_runtime': 164.3063, 'train_samples_per_second': 1.948, 'train_steps_per_second': 0.487, 'total_flos': 1996463454289920.0, 'train_loss': 0.346078397333622})

In [ ]:
# 7. 학습된 모델 저장
output_dir = "./gemma-2-2b-it-finetuned-final"
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('./gemma-2-2b-it-finetuned-final/tokenizer_config.json',
 './gemma-2-2b-it-finetuned-final/special_tokens_map.json',
 './gemma-2-2b-it-finetuned-final/tokenizer.model',
 './gemma-2-2b-it-finetuned-final/added_tokens.json',
 './gemma-2-2b-it-finetuned-final/tokenizer.json')

In [ ]:
# 8. 테스트
# 저장된 모델 로드
test_model = AutoPeftModelForCausalLM.from_pretrained(
    output_dir,
    torch_dtype=torch.bfloat16,
    device_map="auto"
).merge_and_unload()  # LoRA 어댑터 병합

/usr/local/lib/python3.11/dist-packages/accelerate/utils/modeling.py:1569: UserWarning: Current model requires 256 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/accelerate/utils/modeling.py:1569: UserWarning: Current model requires 512 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


In [ ]:
# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 모델을 명시적으로 GPU로 이동
test_model = test_model.to(device)

In [ ]:
# 프롬프트 템플릿 (학습과 일치)
prompt_template = "### 질문: {question}\n### 답변: "

In [ ]:
# 사용자 입력 및 테스트
# user_input = "유튜브 채널 hkcode는 누가 운영하나요?"
user_input = "한국폴리텍대학 스마트금융과는 어디에 위치하나요?"
prompt = prompt_template.format(question=user_input)
inputs = tokenizer(prompt, return_tensors="pt").to(device)

In [ ]:
# 장치 확인
print(f"모델 장치: {next(test_model.parameters()).device}")
print(f"입력 장치: {inputs['input_ids'].device}")

모델 장치: cuda:0
입력 장치: cuda:0


In [ ]:
# 생성 설정 (반복 방지 및 정확성 강화)
outputs = test_model.generate(
    **inputs,
    max_new_tokens=30,  # 답변 길이 줄임
    do_sample=False,
    temperature=0.1,
    repetition_penalty=1.2,  # 반복 방지
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

In [ ]:
# 출력 후처리
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
answer = decoded_output.split("### 답변: ")[-1].strip()  # 답변 부분만 추출
print(f"테스트 출력: {answer}")

테스트 출력: 2019년부터 서울시 강서구 서남동에 위치하고 있습니다. ### 추가정보: 블랙홀을 가로질
